# Setting up your Jupyter Notebook on AWS
So you want to setup your Jupyter notebooks on the cloud?

This tutorial in 4 parts will guide through the process of:
1. Setting up your EC2 instance
2. Generating your security certificate
3. Updating your EC2 box
4. Configuring your Jupyter Notebook

## Sign-in to aws.amazon.com
You will need to create an amazon aws account. If this is the first time that you have registered for AWS services then lucky you! AWS offers a free-tier for the first year to run AWS services (as of the time of this writing - February 2017).

## What is EC2?
EC2 stands for Elastic Cloud Compute and is Amazon Web Services virtual compute environment. It's an on demand service that allows you to provision virtual computers. That is IAAS or infrastructure as a service.

## Setting up your EC2 Instance
_These instructions are the same for Windows, Mac, Ubuntu and Linux users._
Here are the steps you will need to follow to set up your EC2 instance with Jupyter notebooks.
1. Services (top left in navigation bar) > Select EC2
2. Click "Launch instance"
3. Select the most recent Amazon Linux AMI (Amazon Machine Image), it will have the format: Amazon Linux AMI 2016.09.1 (HVM), SSD Volume Type - ami-0b33d91d
  - Most of the required programs will be installed and configured like pip, virtualenv
4. Select the computer environment you want (free-tier includes a T2.Micro environment, for a full list of prices and compute environments see the [link](https://aws.amazon.com/ec2/pricing/on-demand/)) then 
5. Click "Next: Configure Instance Details" (defaults are ok)
6. Click "Next: Add Storage" (defaults are ok unless you are planning on running the notebook up for a long time)
7. Click "Next: Add Tags"
  - Tag your instances
  - Include key-value pairs that will help identify your instance
  - Example "Name: Jupyter Notebook"
8. Click "Configure Security Groups":
  - You will next need to add 3 rules in addition to SSH
  - Click the "Add Rule" button three times and select from the drop down menu the following:
    a) SSH (included by default)
    b) HTTP
    c) HTTPS
    d) Custom TCP Rule - set the port range to: 8888 (this is port through which you will be accessing Jupyter notebook)
 For added security you can set the source to "My IP", however statically ties to your EC2 instance to your IP-address (i.e. geographic location). 
 I only suggest this if you know you're IP won't change/rotate.
9. Click "Review and Launch"
10. Click "Launch"
11. You will be prompted to to use a key-pair value
   - If you don't have a key-pair, create one. 
   - Make sure you store the key=pair somehwere secure since it is your key to access your EC2 instance.
   - Name it something like `jupyter_nb.pem`
12. Once you have downloaded your key-value pair click "Launch Instance"
   - This will bring you to a user prompt window.
   - By clicking "View Instances" at the bottom right of the screen you will be able to track your EC2s boot process

Congratulations you've launched a virtual computer in the cloud!


## Configuring your EC2 Instance to run Jupyter Notebooks
Once your EC2 has been you will need to change the permissions on your downloaded key so that you can tunnel into your cloud instance. You will then need to install software to run Jupyter-notebook in the cloud.

## Changing your pem (security certificate) permissions 
_These instructions are for Mac, Ubuntu and Linux users._

Once you've downloaded you ".pem" file you will need to secure the permissions. 
1. Navigate to the location where your ".pem" file is located
2. Use the command
<code>chmod 400 your_pem_file.pem </code>
  - <code>chmod</code> stands for "Change Mode"
  - By setting the permissions 400 you are allowing read only an not writing 
  - This secures the file from changing it's value
To check that the permission details have changed run <code>ls -la</code> and look for your "*.pem" file.
You should see all the way on the right <code>-r--------</code> which indicates read access only.

## Logging on to your EC2 instance
These instructions are for Mac, Ubuntu and Linux users. 
To log on to your EC2 istance you will need to record your "IPv4 Public IP" address, this will be located in your EC2 console.
If you have navigated away from the webpage "Services > EC2> Running Instances".

Once you have your public IP address logon using the code:
<code> ssh -i ./your_pem_file.pem ec2-user@123.123.123 </code>

The code above assumes you are in the same folder as your "*.pem" if not simply put the correct directory. 
Similarly replace the IP address 123.123.123 with your IPV4 Public IP address.

### Note for Windows users
For Windows instructions to change your pem permissions (using PuttyGen) and loggin-on to your EC2 instance (using Putty) see [here](http://unix.stackexchange.com/questions/116303/convert-amazon-pem-key-to-putty-ppk-key-linux) and [here] (http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/putty.html).

### Updating your system
These instructions are the same for Windows, Mac, Ubuntu and Linux users. 
Once logged in you will next want to set-up your environment this is made up of three steps:
1. Updating your system and downloading appropriate files
2. Configuring your system
3. Modifying jupyter notebook configuration to run securely online (you don't want anyone to touch your files!)


## Updating EC2
```shell
sudo yum update -y
sudo yum install gcc -y
sudo yum install setup-tools
virtualenv venv
source venv/bin/activate
pip install --upgrade pip
pip install -U setuptools
pip install jupyter
```
Next we generate a hashed password in python that will be used to access/protect your notebook. First we launch ipython
```shell
ipython
```
then we generate the hashed password:
```python
from IPython.lib import passwd
# generate password
# copy passwords
passwd()
# output is something like 'sha1:a6400c960e50:227a6a6d13ee418c49b626b7420569996ad64751'
exit()
```


## Configuring your Jupyter notebook environment
Now that we have all the components together we need to configure our Jupyter notebook environment. That means setting up the right configuration settings. The script below generates a "mycert.pem" file required for log on as well as default configuration file for Jupyter notebook. 

```shell
jupyter notebook --generate-config
mkdir certs
cd certs
sudo openssl req -x509 -nodes -days 365 -newkey rsa:1024 -keyout mycert.pem -out mycert.pem (enter all thoughout)
nano /home/ec2-user/.jupyter/jupyter_notebook_config.py
```
The last line of the sequence of code opens up the Jupyter notebook configuration file in nano (a unix text editor). If you are not familiar with nano it is a simple text editor that can be used in shell.
It is here that we will specify the password and "mycert.pem" files to be used.
Without modifying anything in the file paste the following:
```shell
# Kernel config
c.IPKernelApp.pylab = 'inline'  # if you want plotting support always in your notebook

# Notebook config
c.NotebookApp.certfile = u'/home/ec2-user/certs/mycert.pem' #location of your certificate file
c.NotebookApp.ip = '*'
c.NotebookApp.open_browser = False  #so that the ipython notebook does not opens up a browser by default
c.NotebookApp.password = u'sha1:98ff0e580111:12798c72623a6eecd54b51c006b1050f0ac1a62d'  #the encrypted password we generated above
# It is a good idea to put it on a known, fixed port
c.NotebookApp.port = 8888
```
replacing you're hashed password. Once done hit "ctrl+x" which will prompt you whether you want to save or simply exit. Save the file.

## Running your Jupyter Notebooks Persistenty
Next we want to setup Jupyter notebooks in such a way that if you log-off of the temrinal that they persist as a background process. To do this we use the command  <code>screen</code> which let's it run in the backgroud. Paste the following code in your terminal:

```shell
# Create folder for notebooks
cd ~
mkdir Notebooks
cd Notebooks
# Create new screen
screen
# Start Jupyter notebook
jupyter notebook
```
`screen` has useful commands that allow you to navigate across different terminals, they include:

1. Detach from screen: Ctrl-a and then d
2. Create new window: Ctrl-a then c.
3. Switch windows: Ctrl-a then n
4. Reattach to Screen: screen -r

Now copy your IPV4 address (found in the AWS EC2 console), it should look something like: `ec2-54-175-171-107.compute-1.amazonaws.com` where `ec2-54-175-171-107` is your IP-address.

Paste it into your address bar as follows: `https://ec2-54-175-171-107.compute-1.amazonaws.com`. You will need the https. If you're on google chrome you will be prompted to proceed, agree and then enter your password. You should be good to go!


 ### Warning:
 - Remember to shutdown your EC2 enviroment if you are not using it. otherwise you will be charged